In [1]:
import functools
import multiprocessing
from itertools import product

import numpy as np
import scipy.signal as sl

Target:
- get something of shape [Nt, Na, 3]
- compute autocorrelation for each combination of (I, a), (J, b)
- return as [Nt, Na, 3, Na, 3]

In [2]:
# arb. shaped array:
Nt = 4
Na = 2

array = np.ones([Nt, Na, 3])
Nt, *shape = array.shape
Nt, shape

(4, [2, 3])

In [3]:
# create mock data
# fill some data
for (t, I, a) in np.ndindex((Nt, *shape)):
    array[t, I, a] = t + a + I

array

array([[[0., 1., 2.],
        [1., 2., 3.]],

       [[1., 2., 3.],
        [2., 3., 4.]],

       [[2., 3., 4.],
        [3., 4., 5.]],

       [[3., 4., 5.],
        [4., 5., 6.]]])

In [4]:
# reshape to (X, Nt)
data = array.reshape(Nt, -1).swapaxes(0, 1)

data.shape, data

((6, 4), array([[0., 1., 2., 3.],
        [1., 2., 3., 4.],
        [2., 3., 4., 5.],
        [1., 2., 3., 4.],
        [2., 3., 4., 5.],
        [3., 4., 5., 6.]]))

In [5]:
# array returns row for row
for x in data:
    print(x)

# product returns all combinations of rows
for X in product(data[:3], data[:3]):
    print(X)

[0. 1. 2. 3.]
[1. 2. 3. 4.]
[2. 3. 4. 5.]
[1. 2. 3. 4.]
[2. 3. 4. 5.]
[3. 4. 5. 6.]
(array([0., 1., 2., 3.]), array([0., 1., 2., 3.]))
(array([0., 1., 2., 3.]), array([1., 2., 3., 4.]))
(array([0., 1., 2., 3.]), array([2., 3., 4., 5.]))
(array([1., 2., 3., 4.]), array([0., 1., 2., 3.]))
(array([1., 2., 3., 4.]), array([1., 2., 3., 4.]))
(array([1., 2., 3., 4.]), array([2., 3., 4., 5.]))
(array([2., 3., 4., 5.]), array([0., 1., 2., 3.]))
(array([2., 3., 4., 5.]), array([1., 2., 3., 4.]))
(array([2., 3., 4., 5.]), array([2., 3., 4., 5.]))


In [6]:
# apply correlate to each row
def correlate(X, **kwargs):
    Nt = len(X[0])
    return sl.correlate(X[0], X[1], **kwargs)[Nt - 1 :]

# correlate with `fft=True`
def c(X):
    return correlate(X, method='fft')

In [7]:
# reference:
ref_result = np.zeros([Nt, *shape, *shape])
for (I, a) in np.ndindex(*shape):
    for (J, b) in np.ndindex(*shape):
        X = (array[:, I, a], array[:, J, b])
        ref_result[:, I, a, J, b] = c(X)

ref_result.shape

(4, 2, 3, 2, 3)

In [8]:
%%time
result1 = np.zeros((data.shape[0]**2, data.shape[1]))

for ii, X in enumerate(product(data, data)):
    result1[ii] = c(X)

CPU times: user 6.15 ms, sys: 0 ns, total: 6.15 ms
Wall time: 5.5 ms


In [9]:
%%time
# parallelize
with multiprocessing.Pool() as p:
    result2 = np.array(p.map(c, product(data, data)))
    
# to see a speedup, one needs larger data

CPU times: user 11.6 ms, sys: 24.9 ms, total: 36.5 ms
Wall time: 135 ms


In [10]:
np.allclose(result1, result2)

True

In [11]:
# shape back
result = np.moveaxis(result2.reshape((*shape, *shape, Nt)), -1, 0)
result.shape

(4, 2, 3, 2, 3)

In [12]:
np.allclose(result, ref_result)

True